# Using `numba.jit` to speedup the computation of the Euclidean distance matrix 

In this notebook we implement a function to compute the Euclidean distance matrix using Numba's *just-in-time* compilation decorator. We compare it with the NumPy function we wrote before.

We will use two Numba functions here: The decorator ` @numba.jit` and `numba.prange`.

In [1]:
import numpy as np
import numba

In [10]:
@numba.jit(nopython=True)
def euclidean_numba1(x, y):
    """Euclidean square distance matrix using pure loops
    and no NumPy operations
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in numba.prange(num_feat):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r

    return dist_matrix


@numba.jit(nopython=True)
def euclidean_numba2(x, y):
    """Euclidean square distance matrix using loops
    and the `numpy.dot` operation
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in numba.prange(num_samples):
            dist_matrix[i][j] = ((x[i] - y[j])**2).sum()

    return dist_matrix

Let's include here our numpy implementation for comparison.

In [2]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix.
    
    Inputs:
    x: (N, m) numpy array
    y: (N, m) numpy array
    
    Ouput:
    (N, N) Euclidean square distance matrix:
    r_ij = x_ij^2 - y_ij^2
    """

    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T

    xy = np.dot(x, y.T)

    return np.abs(x2 + y2 - 2. * xy)

### Note
Observe that we do the inner loop, which is a reduction, with `numba.prange`. `numba.prange` automatically takes care of data privatization and reductions.

### Exercise 1
Before runing the different functions, could you say which of the two numba implementations would be faster?

In [11]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(euclidean_numpy(a, a) - euclidean_numba1(a, a)).max())
print(np.abs(euclidean_numpy(a, a) - euclidean_numba2(a, a)).max())

3.410605131648481e-13
3.410605131648481e-13


Our Numba implementations can be faster than the NumPy one for a list of small vectors. However, with larger vectors, the NumPy implementation is faster:

In [5]:
nsamples = 100
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

71.2 µs ± 650 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
28.4 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.44 ms ± 36.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
nsamples = 100
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

89 µs ± 8.32 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
427 µs ± 5.75 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.96 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In a more realistic case, our NumPy implementation is much faster:

In [7]:
nsamples = 5000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)

201 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.15 s ± 40.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
